# Doubly periodic flows

This notebook demonstrates the use of the package to simulate the evolution of 2D flows that are doubly periodic (i.e., periodic in both the $x$ and $y$ directions). The flow field quantities are all represented as discrete Fourier transforms (i.e., Fourier series)

$$\omega(x,y,t) = \sum_{k_x = 0}^{N_x-1} \sum_{k_y = 0}^{N_y-1} \hat{\omega}(k_x,k_y,t) e^{i(k_x x + k_y y)}$$

The functions `physical2fourier` and `fourier2physical` take us back and forth between the physical domain and the Fourier domain.

The Navier-Stokes equations are solved in the Fourier domain. The non-linear convective term is handled using a pseudo-spectral technique (i.e., the terms in the product are each transformed to the physical domain, multiplied, and then transformed back, with special care to prevent aliasing errors).

In [ ]:
using PseudoSpectral

In [ ]:
using Plots

### Initialize a grid on $[0,2\pi)\times[0,2\pi)$ 

In [ ]:
# Set the grid dimensions and create points
Nx, Ny = 256, 256
Δx, Δy = 2π/Nx, 2π/Ny
x = range(0,2π,Nx+1)[1:Nx]
y = range(0,2π,Ny+1)[1:Ny]

# Set the viscosity
ν = 0.0005;

### Initialize a vorticity field
There are a few different examples here. Uncomment the one you want.

In [ ]:
# Initial a flow for a perturbed shear layer
w0 = shearlayer(x,y;perturb = 0.02, thickness = 0.2)

# Initialize a flow for two co-rotating vortices
#w0 = twovortex(x,y,distance=1.0)

# Initialize a flow for homogeneous isotropic turbulence. The `factor` controls how strong the turbulence is.
# You can use this to create different turbulent Reynolds numbers (but it has to be reasonably well resolved)
#w0 = randomfield(x,y,factor=5.0)

# You can also add these flows up
#w0 = shearlayer(x,y;perturb=0.0) .+ randomfield(x,y,factor=0.25)


plot(x,y,w0',ratio=1, xlim=(0,2π),ylim=(0,2π))

### Get the initial Fourier coefficients of the field

In [ ]:
what0 = physical2fourier(w0);

# Get the initial velocity field
u, v = velocity(what0);

### Check on the homogeneous isotropic turbulence problem
Here, we compute the initial statistics of the problem. These are only useful for turbulence problems

In [ ]:
# Turbulent kinetic energy
K0 = tke(what0)

# Energy dissipation rate
ϵ0 = dissrate(what0,ν)

# Integral length scale
L0 = K0^(3/2)/ϵ0

# Turbulence Reynolds number
ReL0 = K0^2/ϵ0/ν

# Kolmogorov scale
η0 = L0*ReL0^(-3/4)

# Taylor microscale
lam0 = L0*sqrt(10/ReL0)

# Taylor Reynolds number
Relam0 = (20*ReL0/3)^(1/2)

# Grid spacing relative to Kolmogorov scale.
Δx/lam0

# Grid spacing relative to Kolmogorov scale. It should be reasonably close to 1 for true DNS.
Δx/η0

### Check the Taylor Reynolds number

In [ ]:
Relam0

### Set the time step size

In [ ]:
# For turbulence problem
#Δt = Δx/sqrt(K0)

# For shear layer or vortex problem

Δt = 0.1*Δx/maximum(abs.(u))


params = VorticityFourierFourierNSCache(Nx,Ny,ν,Δt);

### Now, solve the flow
Initialize things first

In [ ]:
t = 0.0
nskip = 50 # Number of steps to run between data saves
nt = 50 # Number of intervals to run (so total simulation time = nt*nskip*Δt)

what = copy(what0)

whistory = [copy(what0)];
thistory = [t];

In [ ]:
# You can re-run this to extend a simulation
for kt in 1:nt
    for jt in 1:nskip
        vorticity_ff_ns_step!(what,params)
        t += Δt
    end
    push!(whistory,copy(what))
    push!(thistory,t)
end

This simulation records the Fourier coefficients of the vorticity field (`what` denotes $\hat{\omega}$). We can use this to compute any other quantity:

In [ ]:
u, v = velocity(what);
ψ = streamfunction(what);
ω = vorticity(what);

In [ ]:
# This is a check that the simulation didn't blow up. (If it is NaN, it blew up!)
maximum(u)

In [ ]:
plot(x,y,ω',ratio=1, xlim=(0,2π),ylim=(0,2π),levels=range(-5,5,length=30))

The history of the Fourier coefficients is recorded in `whistory`, and time is in `thistory`. Let's make a movie:

In [ ]:
@gif for whatj in whistory
    ωj = vorticity(whatj)
    plot(x,y,ωj',ratio=1, xlim=(0,2π),ylim=(0,2π),levels=range(-5,5,length=30),legend=false)
end every 1

### For analysis of homogenous isotropic turbulence
There are several functions that can be run to compute statistics of the turbulent flow. These are

`wavenumbers_fourier(Nx,Ny)` returns the wavenumbers `kx` and `ky`

`energy_fourier(what)` to compute the turbulent kinetic energy in each Fourier mode. This is an array of energies, with `kx` and `ky` as the rows and columns.

`tke(what)` returns a scalar, the total turbulent kinetic energy
      
`dissrate(what,nu)` returns a scalar, the energy dissipation rate

`restress(what)` returns a 2x2 array of the Reynolds stresses

In [ ]:
# Compute the history of the turbulent kinetic energy.
K = [tke(wj) for wj in whistory];

ϵ = [dissrate(wj,ν) for wj in whistory];

In [ ]:
plot(thistory,K,xlabel="\$t\$",ylabel="\$E(t)\$")

## Project
The final project should be written in the form of a technical report, with nice formatting and plots to go along with the explanatory text.

### Project questions:
Simulate the perturbed shear layer problem.
- Show snapshots of the resulting evolution of the shear layer. What is the final behavior?
- Verify that this behavior is not strongly affected by the viscosity. (Again, be careful that you are resolving the flow adequately. For this, check if the solution is different if Nx and Ny are larger.

Simulate decaying homogeneous isotropic 2D turbulence at $\nu = 0.0005$ and different choices of the amplitude of the turbulence (the `factor` argument). Be realistic with this amplitude -- you probably can't simulate turbulence that is too strong, based on grid spacing relative to Kolmogorov.
   - Plot the turbulent kinetic energy and the rate of dissipation vs time, and verify that the rate of change of the TKE is approximately equal to the negative of the dissipation rate.
   - Why does energy decay in this flow?
   - Find the approximate exponent on time to fit a power law for energy vs time (i.e., assuming $E \sim t^p$ and find $p$ approximately)
   - Calculate and plot, as functions of time: the integral length scale, the turbulence Reynolds number, the Taylor microscale length, the Kolmogorov length scale, and the Taylor Reynolds number
   - Explain why the length scales and various Reynolds numbers are becoming larger with time
 
Finally, combine these two flows (see the sum in the examples cell above). Compute the flow with *no* perturbation in the shear layer.
- How does the presence of turbulence affect the evolution of the perturbed shear layer?
- Explain the behavior you observe.